In [11]:
import mysql.connector
import csv
import requests
from bs4 import BeautifulSoup, SoupStrainer
import lxml
import time
import random
import datetime

In [2]:
con = mysql.connector.connect(
    host="localhost", user="root",
    password="", database="dnk")

# create cursor object
cursor = con.cursor()

# assign data query
q_models = "select * from models"

# executing cursor
cursor.execute(q_models)
models = cursor.fetchall()

print(len(models))
print(models[2])


def get_model_id(model_name):
    for record in models:
        if record[1] == model_name:
            return record[0]
    return -1


print(get_model_id('Class'))

621
(3, 'Cabaret', ' ', 1, datetime.datetime(2022, 6, 8, 10, 15, 56), datetime.datetime(2022, 6, 8, 10, 15, 56))
615


In [3]:
read_file = "refs_links.csv"
list_refs_link = []
with open(read_file, 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        list_refs_link.append(row)

In [4]:
print(len(list_refs_link))
chunks = [list_refs_link[x:x + 150] for x in range(0, len(list_refs_link), 150)]
print(len(chunks))
print(chunks[0][0])

33107
221
['https://watchbase.com/a-lange-sohne/1815/236-049']


In [17]:
refs_dict = {}
err_link = []
begin = time.time()
for chunk in chunks[30:40]:
    session = requests.Session()
    for refs_link in chunk:
        try:
            # refs_link = "https://watchbase.com/vacheron-constantin/overseas/2300v-100m-b400"
            header1 = {
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

            header2 = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}

            response = session.get(refs_link[0], headers=random.choice([header1, header2]))
            strainer = SoupStrainer('div', attrs={'class': 'col-md-7'})
            soup = BeautifulSoup(response.content, "lxml", parse_only=strainer)

            # REFS
            model_id = -1
            movement = ' '
            caseMaterial = ' '
            caseBack = ' '
            caseSize = ' '
            bezelMaterial = ' '
            dialMaterial = ' '
            waterResistance = ' '
            # WATCH
            # name = ' '
            # dialColor = ' '
            # yearOfProduction = ' '
            imageLink = ' '

            info_table = soup.select('table.info-table')
            # print(len(info_table))
            frag1 = info_table[0].find_all("tr")
            refs_name = frag1[2].td.text.strip()
            for criteria in frag1:
                if criteria.th.text == "Family:":
                    model_id = get_model_id(criteria.td.text.strip())
                if criteria.th.text == "Movement:":
                    movement = criteria.td.text.strip()

            frag2 = info_table[1].find_all("tr")
            for criteria in frag2:
                if criteria.th.text == "Materials:":
                    caseMaterial = criteria.td.text.strip()
                if criteria.th.text == "Back:":
                    caseBack = criteria.td.text.strip()
                if criteria.th.text == "Diameter:":
                    caseSize = criteria.td.text.strip()
                if criteria.th.text == "Bezel:":
                    bezelMaterial = criteria.td.text.strip()
                if criteria.th.text == "W/R:":
                    waterResistance = criteria.td.text.strip()

            frag3 = info_table[2].find_all("tr")
            for criteria in frag3:
                if criteria.th.text == "Material:":
                    dialMaterial = criteria.td.text.strip()

            watch_image = soup.find('img')
            if watch_image is not None:
                imageLink = watch_image.get('src')

            refs_dict[refs_name] = model_id, movement, caseMaterial, caseBack, caseSize, bezelMaterial, dialMaterial, waterResistance, imageLink
        except:
            err_link.append(refs_link[0])
        time.sleep(random.uniform(0.4, 0.6))

    time.sleep(random.uniform(2, 3))

end = time.time()
print("Process took:", datetime.timedelta(seconds=end-begin))
print("DONE")

Process took: 0:19:57.222738
DONE


In [19]:
# print(refs_name)
# done 0-10
print(len(refs_dict))
print(len(err_link))
# print(err_link[650])

# err_ref_links_file = "err_refs_links.csv"
#
# with open(err_ref_links_file, 'a', encoding='UTF8', newline='') as f:
#     writer = csv.writer(f)
#     for elem in err_link:
#         writer.writerow([elem])

230
1270


In [20]:
refsFile = "refs.csv"

fieldnames_models = ['refsName', 'modelId', 'movement', 'caseMaterial', 'caseBack', 'caseSize', 'bezelMaterial', 'dialMaterial', 'waterResistance', 'imageLink']
# write models
with open(refsFile, 'a', encoding='UTF8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames_models, delimiter=",")
    writer.writeheader()
    rows = list()
    for key, value in refs_dict.items():
        row = {"refsName":key, "modelId":value[0], "movement": value[1], "caseMaterial": value[2], "caseBack": value[3], "caseSize": value[4], "bezelMaterial": value[5], "dialMaterial": value[6], "waterResistance": value[7], "imageLink": value[8]}
        rows.append(row)

    writer.writerows(rows)